In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import os
import shutil
import kagglehub
robikscube_flight_delay_dataset_20182022_path = kagglehub.dataset_download('robikscube/flight-delay-dataset-20182022')

print('Data source import complete.')
robikscube_flight_delay_dataset_20182022_path

100%|██████████| 3.73G/3.73G [01:04<00:00, 61.7MB/s]

Extracting files...


Data source import complete.


'/root/.cache/kagglehub/datasets/robikscube/flight-delay-dataset-20182022/versions/4'

In [1]:
import os
import shutil
import kagglehub

!kaggle datasets download -d robikscube/flight-delay-dataset-20182022
!unzip flight-delay-dataset-20182022.zip

'kaggle' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
%pip install pyspark
%pip install --upgrade pyspark pyarrow

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import os
import pandas as pd
import numpy as np

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import udf, col

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count, round

from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.evaluation import RegressionEvaluator

import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [2]:
spark = SparkSession.builder.master("local[8]") \
                            .appName("Flight Status Prediction") \
                            .config("spark.driver.memory", "4g") \
                            .config("spark.executor.memory", "4g") \
                            .config("spark.memory.storageFraction", "0.5") \
                            .config("spark.memory.fraction", "0.6") \
                            .getOrCreate()
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")


spark
spark.sparkContext

<SparkContext master=local[8] appName=Flight Status Prediction>

In [3]:
# Set Spark configuration to show more columns and adjust the width
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 100)  # Max number of rows to display
spark.conf.set("spark.sql.repl.eagerEval.maxNumCols", 61)  # Max number of columns to display
spark.conf.set("spark.sql.repl.eagerEval.truncate", 0)  # Do not truncate columns

# Import CSV

In [4]:
# Define schema for the flight data
schema = StructType([
    StructField("FlightDate", StringType(), True),
    StructField("Airline", StringType(), True),
    StructField("Origin", StringType(), True),
    StructField("Dest", StringType(), True),
    StructField("Cancelled", StringType(), True),
    StructField("Diverted", StringType(), True),
    StructField("CRSDepTime", StringType(), True),
    StructField("DepTime", StringType(), True),
    StructField("DepDelayMinutes", StringType(), True),
    StructField("DepDelay", StringType(), True),
    StructField("ArrTime", StringType(), True),
    StructField("ArrDelayMinutes", StringType(), True),
    StructField("AirTime", StringType(), True),
    StructField("CRSElapsedTime", StringType(), True),
    StructField("ActualElapsedTime", StringType(), True),
    StructField("Distance", StringType(), True),
    StructField("Year", IntegerType(), True),
    StructField("Quarter", IntegerType(), True),
    StructField("Month", IntegerType(), True),
    StructField("DayofMonth", IntegerType(), True),
    StructField("DayOfWeek", IntegerType(), True),
    StructField("Marketing_Airline_Network", StringType(), True),
    StructField("Operated_or_Branded_Code_Share_Partners", StringType(), True),
    StructField("DOT_ID_Marketing_Airline", StringType(), True),
    StructField("IATA_Code_Marketing_Airline", StringType(), True),
    StructField("Flight_Number_Marketing_Airline", StringType(), True),
    StructField("Operating_Airline", StringType(), True),
    StructField("DOT_ID_Operating_Airline", StringType(), True),
    StructField("IATA_Code_Operating_Airline", StringType(), True),
    StructField("Tail_Number", StringType(), True),
    StructField("Flight_Number_Operating_Airline", StringType(), True),
    StructField("OriginAirportID", IntegerType(), True),
    StructField("OriginAirportSeqID", IntegerType(), True),
    StructField("OriginCityMarketID", IntegerType(), True),
    StructField("OriginCityName", StringType(), True),
    StructField("OriginState", StringType(), True),
    StructField("OriginStateFips", IntegerType(), True),
    StructField("OriginStateName", StringType(), True),
    StructField("OriginWac", IntegerType(), True),
    StructField("DestAirportID", IntegerType(), True),
    StructField("DestAirportSeqID", IntegerType(), True),
    StructField("DestCityMarketID", IntegerType(), True),
    StructField("DestCityName", StringType(), True),
    StructField("DestState", StringType(), True),
    StructField("DestStateFips", IntegerType(), True),
    StructField("DestStateName", StringType(), True),
    StructField("DestWac", IntegerType(), True),
    StructField("DepDel15", StringType(), True),
    StructField("DepartureDelayGroups", StringType(), True),
    StructField("DepTimeBlk", StringType(), True),
    StructField("TaxiOut", StringType(), True),
    StructField("WheelsOff", StringType(), True),
    StructField("WheelsOn", StringType(), True),
    StructField("TaxiIn", StringType(), True),
    StructField("CRSArrTime", StringType(), True),
    StructField("ArrDelay", StringType(), True),
    StructField("ArrDel15", StringType(), True),
    StructField("ArrivalDelayGroups", StringType(), True),
    StructField("ArrTimeBlk", StringType(), True),
    StructField("DistanceGroup", StringType(), True),
    StructField("DivAirportLandings", IntegerType(), True)
])

# Load the flight data
flight_data_path = "/kaggle/input/flight-delay-dataset-20182022/Combined_Flights_2022.csv"
flight_data_path = "./Compressed/Combined_Flights_2022.csv"
Flight_Data = spark.read.csv(flight_data_path, schema=schema, header=True)

# Show the first few rows of the DataFrame
Flight_Data

DataFrame[FlightDate: string, Airline: string, Origin: string, Dest: string, Cancelled: string, Diverted: string, CRSDepTime: string, DepTime: string, DepDelayMinutes: string, DepDelay: string, ArrTime: string, ArrDelayMinutes: string, AirTime: string, CRSElapsedTime: string, ActualElapsedTime: string, Distance: string, Year: int, Quarter: int, Month: int, DayofMonth: int, DayOfWeek: int, Marketing_Airline_Network: string, Operated_or_Branded_Code_Share_Partners: string, DOT_ID_Marketing_Airline: string, IATA_Code_Marketing_Airline: string, Flight_Number_Marketing_Airline: string, Operating_Airline: string, DOT_ID_Operating_Airline: string, IATA_Code_Operating_Airline: string, Tail_Number: string, Flight_Number_Operating_Airline: string, OriginAirportID: int, OriginAirportSeqID: int, OriginCityMarketID: int, OriginCityName: string, OriginState: string, OriginStateFips: int, OriginStateName: string, OriginWac: int, DestAirportID: int, DestAirportSeqID: int, DestCityMarketID: int, DestCi

In [5]:
Flight_Data.columns

['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRS

In [5]:
Flight_Data.show(5)

+----------+--------------------+------+----+---------+--------+----------+-------+---------------+--------+-------+---------------+-------+--------------+-----------------+--------+----+-------+-----+----------+---------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+---------------+------------------+------------------+--------------------+-----------+---------------+---------------+---------+-------------+----------------+----------------+-------------------+---------+-------------+-------------+-------+--------+--------------------+----------+-------+---------+--------+------+----------+--------+--------+------------------+----------+-------------+------------------+
|FlightDate|             Airline|Origin|Dest|Cancelled|Diverted|CRSDepTime|DepTime|DepDelayMinutes|De

In [7]:
# Convert to Pandas DataFrame
pandas_df = Flight_Data.limit(800000).toPandas()  # Limit for performance
import pandas as pd

# Display the Pandas DataFrame
pd.set_option('display.max_columns', None)  # Show all columns
pandas_df

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,1228.0,0.0,40.0,72.0,65.0,212.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4301,C5,20445,C5,N21144,4301,11921,1192102,31921,"Grand Junction, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1100-1159,17.0,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,848.0,0.0,55.0,77.0,80.0,295.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4299,C5,20445,C5,N16170,4299,12206,1220605,32206,"Harlingen/San Benito, TX",TX,48,Texas,74,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,0.0,-1.0,0700-0759,16.0,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,1636.0,0.0,47.0,70.0,82.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4298,C5,20445,C5,N21144,4298,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1500-1559,21.0,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,1547.0,0.0,57.0,90.0,77.0,376.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4296,C5,20445,C5,N11184,4296,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,11973,1197302,31973,"Gulfport/Biloxi, MS",MS,28,Mississippi,53,0.0,-1.0,1400-1459,16.0,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,1251.0,6.0,49.0,70.0,76.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4295,C5,20445,C5,N17146,4295,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,0.0,1100-1159,19.0,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,2022-02-01,Capital Cargo International,CLT,MOB,False,False,931,922.0,0.0,-9.0,1050.0,2.0,77.0,137.0,148.0,526.0,2022,1,2,1,2,AA,AA_CODESHARE,19805,AA,6187,PT,20427,PT,N626AE,6187,11057,1105703,31057,"Charlotte, NC",NC,37,North Carolina,36,13422,1342202,30562,"Mobile, AL",AL,1,Alabama,51,0.0,-1.0,0900-0959,32.0,954.0,1011.0,39.0,1048,2.0,0.0,0.0,1000-1059,3,0
799996,2022-02-02,Capital Cargo International,CLT,MOB,False,False,931,921.0,0.0,-10.0,1024.0,0.0,85.0,137.0,123.0,526.0,2022,1,2,2,3,AA,AA_CODESHARE,19805,AA,6187,PT,20427,PT,N682AE,6187,11057,1105703,31057,"Charlotte, NC",NC,37,North Carolina,36,13422,1342202,30562,"Mobile, AL",AL,1,Alabama,51,0.0,-1.0,0900-0959,34.0,955.0,1020.0,4.0,1048,-24.0,0.0,-2.0,1000-1059,3,0
799997,2022-02-03,Capital Cargo International,CLT,MOB,False,False,933,1155.0,142.0,142.0,1248.0,116.0,89.0,139.0,113.0,526.0,2022,1,2,3,4,AA,AA_CODESHARE,19805,AA,6187,PT,20427,PT,N649PP,6187,11057,1105703,31057,"Charlotte, NC",NC,37,North Carolina,36,1

----

# EDA

In [8]:
pandas_df.describe()

,Year,Quarter,Month,DayofMonth,DayOfWeek,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginStateFips,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestStateFips,DestWac,DivAirportLandings
count,800000.0,800000.000000,800000.000000,800000.000000,800000.000000,800000.000000,8.000000e+05,800000.000000,800000.000000,800000.000000,800000.000000,8.000000e+05,800000.000000,800000.000000,800000.000000,800000.000000
mean,2022.0,1.725362,3.450725,15.075789,4.032382,12683.014331,1.268305e+06,31763.395531,27.272164,55.341110,12685.498060,1.268554e+06,31764.745071,27.268941,55.348786,0.003544
std,0.0,0.446332,0.892663,8.482178,1.974182,1525.848018,1.525846e+05,1327.326244,16.791962,26.513359,1526.933492,1.526932e+05,1327.811601,16.795435,26.516991,0.117967
min,2022.0,1.000000,2.000000,1.000000,1.000000,10135.000000,1.013506e+06,30070.000000,1.000000,1.000000,10135.000000,1.013506e+06,30070.000000,1.000000,1.000000,0.000000
25%,2022.0,1.000000,2.000000,8.000000,2.000000,11292.000000,1.129202e+06,30693.000000,12.000000,34.000000,11292.000000,1.129202e+06,30693.000000,12.000000,34.000000,0.000000
50%,2022.0,2.000000,4.000000,15.000000,4.000000,12889.000000,1.288903e+06,31453.000000,26.000000,52.000000,12889.000000,1.288903e+06,31453.000000,26.000000,52.000000,0.000000
75%,2022.0,2.000000,4.000000,22.000000,6.000000,14057.000000,1.405702e+06,32575.000000,42.000000,82.000000,14057.000000,1.405702e+06,32575.000000,42.000000,82.000000,0.000000
max,2022.0,2.000000,4.000000,30.000000,7.000000,16869.000000,1.686901e+06,36101.000000,78.000000,93.000000,16869.000000,1.686901e+06,36101.000000,78.000000,93.000000,9.000000


## Check NAN

In [9]:
import pandas as pd

# Set Pandas display options to show all columns
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None)     # Show all rows if necessary
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

# Display the count of missing values for each column
print(pandas_df.isnull().sum())

FlightDate                                     0
Airline                                        0
Origin                                         0
Dest                                           0
Cancelled                                      0
Diverted                                       0
CRSDepTime                                     0
DepTime                                    24764
DepDelayMinutes                            24778
DepDelay                                   24778
ArrTime                                    25462
ArrDelayMinutes                            27037
AirTime                                    27037
CRSElapsedTime                                 0
ActualElapsedTime                          27037
Distance                                       0
Year                                           0
Quarter                                        0
Month                                          0
DayofMonth                                     0
DayOfWeek           

In [10]:
import pandas as pd

# Set Pandas display options
pd.set_option('display.max_columns', None)  # Show all columns

# Check for missing values and filter to show only columns with nulls
missing_values = pandas_df.isnull().sum()
missing_values = missing_values[missing_values > 0]  # Filter to keep only columns with nulls

# Display the missing values
print("Columns with Missing Values:")
print(missing_values)

Columns with Missing Values:
DepTime                 24764
DepDelayMinutes         24778
DepDelay                24778
ArrTime                 25462
ArrDelayMinutes         27037
AirTime                 27037
ActualElapsedTime       27037
Tail_Number              3466
DepDel15                24778
DepartureDelayGroups    24778
TaxiOut                 25136
WheelsOff               25136
WheelsOn                25463
TaxiIn                  25463
ArrDelay                27037
ArrDel15                27037
ArrivalDelayGroups      27037
dtype: int64


In [11]:
# Count the number of rows in the DataFrame
row_count = Flight_Data.count()

# Print the number of rows
print(f"Number of rows in the DataFrame: {row_count}")

Number of rows in the DataFrame: 4078318


Dikarenakan ada sedikit jadinya kita tidak perlu khawatir ini akan mempengaruhi data. datasetnya ada 4078318 rows, jadi ini dari 400000 yang NAN itu 10347 atau sekitar 0.26%

## Charts

In [12]:
import plotly.express as px
import pandas as pd

# Convert DepDelayMinutes to numeric, coercing errors to NaN
pandas_df['DepDelayMinutes'] = pd.to_numeric(pandas_df['DepDelayMinutes'], errors='coerce')

# Filter for delays greater than 30 minutes
filtered_df = pandas_df[pandas_df['DepDelayMinutes'] > 30]

# Create the histogram
fig = px.histogram(filtered_df, x='DepDelayMinutes', nbins=50, title='Distribution Lama Delays Keberangkatan (> 30 Menit)')
fig.update_layout(xaxis_title='Departure Delay (minutes)', yaxis_title='Frequency')
fig.show()

In [13]:
pandas_df['Cancelled'] = pandas_df['Cancelled'].map({'True': True, 'False': False})
pandas_df['Cancelled'] = pandas_df['Cancelled'].astype(int)

monthly_cancellations = pandas_df.groupby('Month')['Cancelled'].sum().reset_index()

fig = px.bar(monthly_cancellations, x='Month', y='Cancelled', title='Banyak Penerbangan dibatalkan berdasarkan bulan')
fig.update_layout(xaxis_title='Month', yaxis_title='Number of Cancellations')
fig.show()

In [14]:
import pandas as pd
import plotly.express as px

pandas_df['DepDelayMinutes'] = pd.to_numeric(pandas_df['DepDelayMinutes'], errors='coerce')

# Create the DelayGroup column
pandas_df["DelayGroup"] = None

# Categorizing delays
pandas_df.loc[pandas_df["DepDelayMinutes"] == 0, "DelayGroup"] = "OnTime_Early"
pandas_df.loc[(pandas_df["DepDelayMinutes"] > 0) & (pandas_df["DepDelayMinutes"] <= 15), "DelayGroup"] = "Small_Delay"
pandas_df.loc[(pandas_df["DepDelayMinutes"] > 15) & (pandas_df["DepDelayMinutes"] <= 30), "DelayGroup"] = "Medium_Delay"
pandas_df.loc[pandas_df["DepDelayMinutes"] > 30, "DelayGroup"] = "Large_Delay"
pandas_df.loc[pandas_df["Cancelled"] == 1, "DelayGroup"] = "Cancelled"  # Assuming 1 for canceled flights

# Count occurrences of each DelayGroup
delay_counts = pandas_df["DelayGroup"].value_counts().reset_index()
delay_counts.columns = ['DelayGroup', 'Count']

# Plotting with Plotly Express
fig = px.bar(delay_counts, x='Count', y='DelayGroup',
              title='Flight Delay Categories',
              labels={'Count': 'Frequency', 'DelayGroup': 'Delay Group'},
              orientation='h')

fig.update_layout(xaxis_title='Frequency', yaxis_title='Delay Group')
fig.show()

Dikarenakan pada 200000 data pertama adalah isinya bulan 4 semua maka kita hanya dapat mengetahui bahwa yang cancel pada bulan itu ada 3588 penerbangan

In [15]:
# Filter for canceled flights
canceled_flights = pandas_df[pandas_df['Cancelled'] == 1]

# Optionally, display the number of canceled flights
print(f"Total number of canceled flights: {len(canceled_flights)}")

Total number of canceled flights: 25251


ini EDA di limit jadi 1000 data pertama untuk performa. namun begitu hanya ada 3 penerbangan yang di cancel sedangkan penerbangan yang dikencel sebenarnya ada ~123000 penerbangan.

In [16]:
# Check the data types of the relevant columns
print(pandas_df[['DepDelayMinutes', 'ArrDelayMinutes']].dtypes)

# Convert columns to numeric, coercing errors to NaN
pandas_df['DepDelayMinutes'] = pd.to_numeric(pandas_df['DepDelayMinutes'], errors='coerce')
pandas_df['ArrDelayMinutes'] = pd.to_numeric(pandas_df['ArrDelayMinutes'], errors='coerce')

# Now calculate the average delay by airline
avg_delay_airline = pandas_df.groupby('Airline')[['DepDelayMinutes', 'ArrDelayMinutes']].mean().reset_index()

# Create the bar plot
fig = px.bar(avg_delay_airline, x='Airline', y='DepDelayMinutes', title='Rata-Rata Departure Delay oleh Airline')
fig.update_layout(xaxis_title='Airline', yaxis_title='Average Departure Delay (minutes)', xaxis_tickangle=0)
fig.show()

DepDelayMinutes    float64
ArrDelayMinutes     object
dtype: object


In [17]:
# Convert the DepDelayMinutes to numeric, coercing errors to NaN
pandas_df['DepDelayMinutes'] = pd.to_numeric(pandas_df['DepDelayMinutes'], errors='coerce')

# Create a pivot table for delays
delay_pivot = pandas_df.pivot_table(values='DepDelayMinutes', index='DayOfWeek', columns='Month', aggfunc='mean')

fig = px.imshow(delay_pivot, title='Average Departure Delay Heatmap by Day of Week and Month',
                labels={'x': 'Month', 'y': 'Day of Week', 'color': 'Average Delay (minutes)'},
                x=delay_pivot.columns, y=delay_pivot.index)
fig.show()

Dikarenakan dataset 200000 pertama hanya include bulan 4 maka kita hanya mendapatkan bulan 4 saja.

In [18]:
# Convert FlightDate to datetime if it isn't already
pandas_df['FlightDate'] = pd.to_datetime(pandas_df['FlightDate'], errors='coerce')

# Create a new column for Year
pandas_df["Year"] = pandas_df["FlightDate"].dt.year

# Group by Year and calculate the normalized counts of DelayGroup
df_agg = pandas_df.groupby("Year")["DelayGroup"].value_counts(normalize=True).unstack() * 100

# Define the column order
col_order = ["OnTime_Early", "Small_Delay", "Medium_Delay", "Large_Delay", "Cancelled"]

# Display the aggregated DataFrame with a gradient style
df_agg[col_order].style.background_gradient(cmap="Greens")

DelayGroup,OnTime_Early,Small_Delay,Medium_Delay,Large_Delay,Cancelled
Year,,,,,
2022,58.921625,17.978125,7.042875,12.901000,3.156375


# Data Preprocessing

dikarenakan kita tidak mempedulikan missing value maka kita coba untuk mengubah FlightDate jadi di pisah agar lebih mudah di process

In [19]:
# Convert 'FlightDate' to datetime
pandas_df['FlightDate'] = pd.to_datetime(pandas_df['FlightDate'], errors='coerce')

# Drop rows where 'FlightDate' could not be converted
pandas_df.dropna(subset=['FlightDate'], inplace=True)

# Step 3: Create new features
pandas_df['Year'] = pandas_df['FlightDate'].dt.year
pandas_df['Month'] = pandas_df['FlightDate'].dt.month
pandas_df['DayofWeek'] = pandas_df['FlightDate'].dt.dayofweek  # 0=Monday, 6=Sunday

# Show the result
print(pandas_df[['FlightDate', 'Year', 'Month', 'DayofWeek']].head())

  FlightDate  Year  Month  DayofWeek
0 2022-04-04  2022      4          0
1 2022-04-04  2022      4          0
2 2022-04-04  2022      4          0
3 2022-04-04  2022      4          0
4 2022-04-04  2022      4          0


string catagorical di encode

In [20]:
# Convert the 'Cancelled' column to boolean, if it's in string format
pandas_df['Cancelled'] = pandas_df['Cancelled'].replace({'True': True, 'False': False})

# Now convert to integer
pandas_df['Cancelled'] = pandas_df['Cancelled'].astype(int)

print(pandas_df['Cancelled'].head())

0    0
1    0
2    0
3    0
4    0
Name: Cancelled, dtype: int32


In [21]:
import pandas as pd

# List of columns to exclude
exclude_cols = [
    'FlightDate', 'Origin', 'Dest', 'Tail_Number', 'DepTimeBlk',
    'OriginCityName', 'OriginState', 'IATA_Code_Marketing_Airline',
    'OriginStateName', 'DestCityName', 'Operating_Airline', 'IATA_Code_Operating_Airline',
    'DestState', 'DestStateName', 'Airline', 'Operated_or_Branded_Code_Share_Partners'
]

# Get all columns in the DataFrame
all_cols = pandas_df.columns

# Determine which columns to convert to int
cols_to_convert = [col for col in all_cols if col not in exclude_cols]

# Convert the specified columns to int, handling non-numeric values
for col in cols_to_convert:
    try:
        # Convert to numeric, forcing errors to NaN
        pandas_df[col] = pd.to_numeric(pandas_df[col], errors='coerce')

        # Now convert to int, filling NaN with a specific value if needed (e.g., 0)
        pandas_df[col] = pandas_df[col].fillna(0).astype(int)

    except Exception as e:
        print(f"Error converting column {col}: {e}")

# Display the DataFrame to check the changes
print(pandas_df.head())

  FlightDate                                    Airline Origin Dest  Cancelled  Diverted  CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ArrTime  ArrDelayMinutes  AirTime  CRSElapsedTime  ActualElapsedTime  Distance  Year  Quarter  Month  DayofMonth  DayOfWeek  Marketing_Airline_Network Operated_or_Branded_Code_Share_Partners  DOT_ID_Marketing_Airline IATA_Code_Marketing_Airline  Flight_Number_Marketing_Airline Operating_Airline  DOT_ID_Operating_Airline IATA_Code_Operating_Airline Tail_Number  Flight_Number_Operating_Airline  OriginAirportID  OriginAirportSeqID  OriginCityMarketID            OriginCityName OriginState  OriginStateFips OriginStateName  OriginWac  DestAirportID  DestAirportSeqID  DestCityMarketID         DestCityName DestState  DestStateFips DestStateName  DestWac  DepDel15  DepartureDelayGroups DepTimeBlk  TaxiOut  WheelsOff  WheelsOn  TaxiIn  CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTimeBlk  DistanceGroup  DivAirportLandings  DelayGroup  DayofWeek
0 202

In [22]:
from sklearn.preprocessing import LabelEncoder

object_cols = int_columns = [
    'Origin', 'Dest', 'Tail_Number', 'DepTimeBlk',
    'OriginCityName', 'OriginState', 'IATA_Code_Marketing_Airline',
    'OriginStateName', 'DestCityName', 'Operating_Airline', 'IATA_Code_Operating_Airline',
    'DestState', 'DestStateName', 'Airline', 'Operated_or_Branded_Code_Share_Partners'
]

# Membuat objek LabelEncoder
label_encoder = LabelEncoder()

# Mengencoding setiap kolom bertipe object
for col in object_cols:
    pandas_df[col] = label_encoder.fit_transform(pandas_df[col])

# Menampilkan DataFrame yang telah diencoding
print(pandas_df.head())

  FlightDate  Airline  Origin  Dest  Cancelled  Diverted  CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ArrTime  ArrDelayMinutes  AirTime  CRSElapsedTime  ActualElapsedTime  Distance  Year  Quarter  Month  DayofMonth  DayOfWeek  Marketing_Airline_Network  Operated_or_Branded_Code_Share_Partners  DOT_ID_Marketing_Airline  IATA_Code_Marketing_Airline  Flight_Number_Marketing_Airline  Operating_Airline  DOT_ID_Operating_Airline  IATA_Code_Operating_Airline  Tail_Number  Flight_Number_Operating_Airline  OriginAirportID  OriginAirportSeqID  OriginCityMarketID  OriginCityName  OriginState  OriginStateFips  OriginStateName  OriginWac  DestAirportID  DestAirportSeqID  DestCityMarketID  DestCityName  DestState  DestStateFips  DestStateName  DestWac  DepDel15  DepartureDelayGroups  DepTimeBlk  TaxiOut  WheelsOff  WheelsOn  TaxiIn  CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTimeBlk  DistanceGroup  DivAirportLandings  DelayGroup  DayofWeek
0 2022-04-04        6     138    94         

In [23]:
print(pandas_df.head())

  FlightDate  Airline  Origin  Dest  Cancelled  Diverted  CRSDepTime  DepTime  DepDelayMinutes  DepDelay  ArrTime  ArrDelayMinutes  AirTime  CRSElapsedTime  ActualElapsedTime  Distance  Year  Quarter  Month  DayofMonth  DayOfWeek  Marketing_Airline_Network  Operated_or_Branded_Code_Share_Partners  DOT_ID_Marketing_Airline  IATA_Code_Marketing_Airline  Flight_Number_Marketing_Airline  Operating_Airline  DOT_ID_Operating_Airline  IATA_Code_Operating_Airline  Tail_Number  Flight_Number_Operating_Airline  OriginAirportID  OriginAirportSeqID  OriginCityMarketID  OriginCityName  OriginState  OriginStateFips  OriginStateName  OriginWac  DestAirportID  DestAirportSeqID  DestCityMarketID  DestCityName  DestState  DestStateFips  DestStateName  DestWac  DepDel15  DepartureDelayGroups  DepTimeBlk  TaxiOut  WheelsOff  WheelsOn  TaxiIn  CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTimeBlk  DistanceGroup  DivAirportLandings  DelayGroup  DayofWeek
0 2022-04-04        6     138    94         

Normalisasikan data

In [29]:
numerical_features = pandas_df.select_dtypes(include=['float64', 'int64', 'int32']).columns.tolist()

print("Numerical features:", numerical_features)

Numerical features: ['FlightDate', 'Airline', 'Origin', 'Dest', 'Cancelled', 'Diverted', 'CRSDepTime', 'DepTime', 'DepDelayMinutes', 'DepDelay', 'ArrTime', 'ArrDelayMinutes', 'AirTime', 'CRSElapsedTime', 'ActualElapsedTime', 'Distance', 'Year', 'Quarter', 'Month', 'DayofMonth', 'DayOfWeek', 'Marketing_Airline_Network', 'Operated_or_Branded_Code_Share_Partners', 'DOT_ID_Marketing_Airline', 'IATA_Code_Marketing_Airline', 'Flight_Number_Marketing_Airline', 'Operating_Airline', 'DOT_ID_Operating_Airline', 'IATA_Code_Operating_Airline', 'Tail_Number', 'Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginAirportSeqID', 'OriginCityMarketID', 'OriginCityName', 'OriginState', 'OriginStateFips', 'OriginStateName', 'OriginWac', 'DestAirportID', 'DestAirportSeqID', 'DestCityMarketID', 'DestCityName', 'DestState', 'DestStateFips', 'DestStateName', 'DestWac', 'DepDel15', 'DepartureDelayGroups', 'DepTimeBlk', 'TaxiOut', 'WheelsOff', 'WheelsOn', 'TaxiIn', 'CRSArrTime', 'ArrDelay', 'ArrDel15', 

In [30]:
from sklearn.preprocessing import MinMaxScaler

# Mencari semua kolom yang numerik
numerical_features = pandas_df.select_dtypes(include=['float64', 'int64', 'int32']).columns.tolist()

# Inisialisasi scaler
scaler = MinMaxScaler()

# Normalisasi data
pandas_df[numerical_features] = scaler.fit_transform(pandas_df[numerical_features])

print("Data setelah normalisasi:")
print(pandas_df[numerical_features].head())

Data setelah normalisasi:
   FlightDate  Airline    Origin      Dest  Cancelled  Diverted  CRSDepTime   DepTime  DepDelayMinutes  DepDelay   ArrTime  ArrDelayMinutes   AirTime  CRSElapsedTime  ActualElapsedTime  Distance  Year  Quarter  Month  DayofMonth  DayOfWeek  Marketing_Airline_Network  Operated_or_Branded_Code_Share_Partners  DOT_ID_Marketing_Airline  IATA_Code_Marketing_Airline  Flight_Number_Marketing_Airline  Operating_Airline  DOT_ID_Operating_Airline  IATA_Code_Operating_Airline  Tail_Number  Flight_Number_Operating_Airline  OriginAirportID  OriginAirportSeqID  OriginCityMarketID  OriginCityName  OriginState  OriginStateFips  OriginStateName  OriginWac  DestAirportID  DestAirportSeqID  DestCityMarketID  DestCityName  DestState  DestStateFips  DestStateName   DestWac  DepDel15  DepartureDelayGroups  DepTimeBlk   TaxiOut  WheelsOff  WheelsOn    TaxiIn  CRSArrTime  ArrDelay  ArrDel15  ArrivalDelayGroups  ArrTimeBlk  DistanceGroup  DivAirportLandings  DelayGroup  DayofWeek
0   

In [31]:
# Ensure 'FlightDate' is in datetime format
pandas_df['FlightDate'] = pd.to_datetime(pandas_df['FlightDate'])

# Convert 'FlightDate' to Unix timestamp
pandas_df['FlightDate'] = pandas_df['FlightDate'].astype('int64') // 10**9  # Convert to seconds

# Check the first few rows to confirm
print(pandas_df[['FlightDate']].head())

   FlightDate
0           0
1           0
2           0
3           0
4           0


In [32]:
pandas_df.dtypes

FlightDate                                   int64
Airline                                    float64
Origin                                     float64
Dest                                       float64
Cancelled                                  float64
Diverted                                   float64
CRSDepTime                                 float64
DepTime                                    float64
DepDelayMinutes                            float64
DepDelay                                   float64
ArrTime                                    float64
ArrDelayMinutes                            float64
AirTime                                    float64
CRSElapsedTime                             float64
ActualElapsedTime                          float64
Distance                                   float64
Year                                       float64
Quarter                                    float64
Month                                      float64
DayofMonth                     

# Classification, Evaluation

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

#definisikan target feature & variable
X = pandas_df.drop('Cancelled', axis=1)
y = pandas_df['Cancelled']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

Train RandomForestClassifier

In [44]:
# Initialize the Random Forest Classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [37]:
from sklearn.svm import SVC

# Initiate Model
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifier = SVC(kernel='linear', random_state=42)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [39]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=14)

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=14)

# Buat prediksi dan evaluasi

## RandomForestClassifier

In [45]:
# Make predictions
RF_Pred = rf_model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, RF_Pred))

# Confusion matrix
cm = confusion_matrix(y_test, RF_Pred)
print("Confusion Matrix:")
print(cm)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      1.00      1.00     10242

    accuracy                           1.00    320000
   macro avg       1.00      1.00      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309758      0]
 [     0  10242]]


In [46]:
# Classification Report
report = classification_report(y_test, RF_Pred, output_dict=True)

print("Classification Report:")
print(classification_report(y_test, RF_Pred))

# Confusion Matrix
RF = confusion_matrix(y_test, RF_Pred)
print("Confusion Matrix:")
print(RF)

# Extract Precision and Recall for each class
for label, metrics in report.items():
    if label != 'accuracy':  # Exclude accuracy
        precision = metrics['precision']
        recall = metrics['recall']
        print(f"Class {label} - Precision: {precision:.4f}, Recall: {recall:.4f}")

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      1.00      1.00     10242

    accuracy                           1.00    320000
   macro avg       1.00      1.00      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309758      0]
 [     0  10242]]
Class 0.0 - Precision: 1.0000, Recall: 1.0000
Class 1.0 - Precision: 1.0000, Recall: 1.0000
Class macro avg - Precision: 1.0000, Recall: 1.0000
Class weighted avg - Precision: 1.0000, Recall: 1.0000


## SVN classifier

In [47]:
# Make predictions
SVN_pred = classifier.predict(X_test)

# Evalute the model
print("Classification Report:")
print(classification_report(y_test, SVN_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, SVN_pred)
print("Confusion Matrix:")
print(cm)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      1.00      1.00     10242

    accuracy                           1.00    320000
   macro avg       1.00      1.00      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309757      1]
 [     0  10242]]


In [48]:
# Classification Report
report = classification_report(y_test, SVN_pred, output_dict=True)

print("Classification Report:")
print(classification_report(y_test, SVN_pred))

# Confusion Matrix
knn_cm = confusion_matrix(y_test, SVN_pred)
print("Confusion Matrix:")
print(knn_cm)

# Extract Precision and Recall for each class
for label, metrics in report.items():
    if label != 'accuracy':  # Exclude accuracy
        precision = metrics['precision']
        recall = metrics['recall']
        print(f"Class {label} - Precision: {precision:.4f}, Recall: {recall:.4f}")

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      1.00      1.00     10242

    accuracy                           1.00    320000
   macro avg       1.00      1.00      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309757      1]
 [     0  10242]]
Class 0.0 - Precision: 1.0000, Recall: 1.0000
Class 1.0 - Precision: 0.9999, Recall: 1.0000
Class macro avg - Precision: 1.0000, Recall: 1.0000
Class weighted avg - Precision: 1.0000, Recall: 1.0000


## KKN

In [40]:
knn_y_pred = knn.predict(X_test)

print("Classification Report:")
print(classification_report(y_test, knn_y_pred))

# Confusion Matrix
knn_cm = confusion_matrix(y_test, knn_y_pred)
print("Confusion Matrix:")
print(knn_cm)

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      0.99      0.99     10242

    accuracy                           1.00    320000
   macro avg       1.00      0.99      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309757      1]
 [   147  10095]]


In [41]:
# Classification Report
report = classification_report(y_test, knn_y_pred, output_dict=True)

print("Classification Report:")
print(classification_report(y_test, knn_y_pred))

# Confusion Matrix
knn_cm = confusion_matrix(y_test, knn_y_pred)
print("Confusion Matrix:")
print(knn_cm)

# Extract Precision and Recall for each class
for label, metrics in report.items():
    if label != 'accuracy':  # Exclude accuracy
        precision = metrics['precision']
        recall = metrics['recall']
        print(f"Class {label} - Precision: {precision:.4f}, Recall: {recall:.4f}")

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    309758
         1.0       1.00      0.99      0.99     10242

    accuracy                           1.00    320000
   macro avg       1.00      0.99      1.00    320000
weighted avg       1.00      1.00      1.00    320000

Confusion Matrix:
[[309757      1]
 [   147  10095]]
Class 0.0 - Precision: 0.9995, Recall: 1.0000
Class 1.0 - Precision: 0.9999, Recall: 0.9856
Class macro avg - Precision: 0.9997, Recall: 0.9928
Class weighted avg - Precision: 0.9995, Recall: 0.9995
